In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import gender_guesser.detector as gender
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In order for the module gender-guesser to work, one must install the module using Kaggle's console with internet option switched on and type "!pip install gender-guesser". Only then you can import gender_guesser.detector as gender code.

In [ ]:
gd = gender.Detector()
df = pd.read_csv(r"/kaggle/input/golden-globe-awards/golden_globe_awards.csv")

In [ ]:
df.info()

In [ ]:
genders = []
for i in df.category:
    if "actor" in i.lower():
        genders.append("male")
    elif "actress" in i.lower():
        genders.append("female")
    else:
        genders.append("null")
df['genders'] = genders
df.head()

Added new column to justify if nominees are male or female by using keywords such as "Actor" or "Actress" from category, those that did not mention will be justify "null" for now. The next is justify "null" to male or female using gender-guesser module.

In [ ]:
df.genders.value_counts()

In [ ]:
nm = df.nominee.str.split()
firstname = []
for i in range(len(nm)):
    firstname.append(nm[i][0])

This is to point out the first name of all nominees, nominees with movie title only will later justified as unknown using gender-guesser module.

In [ ]:
guess = []
for i in range(len(firstname)):
    guess.append(gd.get_gender(firstname[i]))    # assign gender from all the first name into guess's list.
    
for i in range(len(df)):
    if "null" in df.genders[i]:
        df.genders[i] = df.genders[i].replace("null", guess[i])    # replace the nulls from df.genders to the assigned guess's list.
        
df.genders.value_counts()

from previous df.genders, the output was:

* null      3545
* female    2233
* male      2213


After going through the gender-guesser function, the new df.genders output is:

* male             3387
* female           2452
* unknown          1813
* mostly_female     204
* mostly_male        76
* andy               59

(andy means androgynous, and unknown are mostly due to gender-guesser read the title of the movie instead of celebrity first name)

In [ ]:
df.genders = df.genders.replace("mostly_female", "female")
df.genders = df.genders.replace("mostly_male", "male")

Lastly, turns all the mostly_female/male to female and male to sums up the male and female counts, assuming they are mostly male and female names.

In [ ]:
df.genders.value_counts().plot.pie(autopct='%1.1f%%')

In [ ]:
year = year = df[(df.genders == 'male')| (df.genders == 'female')]
year = year.groupby('year_film').genders.value_counts()
year.unstack().plot.line()

the "year" value is created to count numbers of nominated male and female nominees from year 1943 to 2019. The plot is shown above. The "andy" and "unknown" category is taken off for clear picture understanding.

In [ ]:
win = df[(df.genders == 'male')| (df.genders == 'female')]
win = win[win.win == True]
win = win.groupby('year_film').genders.value_counts()
win.unstack().plot.line()


the "win" value is determine the award winner for both male and female nominess from 1943 to 2019. The "andy" and "unknown" category is taken off for clear picture understanding.